In [2]:
pip install scikit-fuzzy ipywidgets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.8/920.8 kB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.3 MB/s eta 0:00:00


In [3]:
# ================================================================
# 100 CLIENTES + LÓGICA DIFUSA PARA PROBABILIDAD DE PRÉSTAMO (1..100)
# - Genera dataset de 100 clientes (CSV)
# - Construye sistema fuzzy (entradas categóricas + numéricas)
# - Evalúa fila a fila y guarda CSV con 'prob_prestamo'
# ================================================================

import numpy as np
import pandas as pd
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import os

# ------------------------------
# 0) Parámetros y utilidades
# ------------------------------
np.random.seed(42)  # reproducibilidad
csv_in  = "clientes_100.csv"
csv_out = "clientes_100_con_prob.csv"

# Mapa si/no a 0/1 (acepta variantes)
map_si_no = {"si":1, "sí":1, "SI":1, "Sí":1, "Si":1, "sI":1,
             "no":0, "NO":0, "No":0, "nO":0}

# ------------------------------
# 1) Generar base sintética (100 clientes)
# ------------------------------
if not os.path.exists(csv_in):
    # Probabilidades para variables categóricas (ajústalas si quieres otro mix)
    p_casa = 0.55          # prob. de "si" en tiene_casa
    p_resp = 0.50          # prob. de "si" en es_responsable
    p_cred = 0.35          # prob. de "si" en creditos_activos

    # Categóricas: 'si'/'no'
    tiene_casa = np.where(np.random.rand(100) < p_casa, "si", "no")
    es_resp    = np.where(np.random.rand(100) < p_resp, "si", "no")
    cred_act   = np.where(np.random.rand(100) < p_cred, "si", "no")

    # Numéricas (en millones) – puedes cambiar la distribución a conveniencia
    # Salario: uniforme entre 2 y 10
    salario = np.round(np.random.uniform(2.0, 10.0, size=100), 2)
    # Gastos: correlacionados levemente con salario (para parecer realistas)
    gastos_base = np.clip(salario/3 + np.random.normal(0, 0.3, size=100), 2.0, 5.0)
    gastos = np.round(gastos_base, 2)

    df_gen = pd.DataFrame({
        "tiene_casa":       tiene_casa,
        "es_responsable":   es_resp,
        "salario_millones": salario,
        "gastos_millones":  gastos,
        "creditos_activos": cred_act
    })
    df_gen.to_csv(csv_in, index=False)
    print(f"✅ Dataset sintético generado: {csv_in}")
else:
    print(f"ℹ️ Ya existe {csv_in}; no se regenera.")

# ------------------------------
# 2) Universos de discurso (rangos donde viven las membresías)
# ------------------------------
u_categ = np.array([0, 1])                    # categóricas (0=no, 1=si)
u_sal   = np.arange(2.0, 10.0 + 0.1, 0.1)     # salario (2..10)
u_gast  = np.arange(2.0, 5.0  + 0.1, 0.1)     # gastos  (2..5)
u_prob  = np.arange(1.0, 100.0 + 1.0, 1.0)    # prob. préstamo (1..100)

# ------------------------------
# 3) Variables difusas (Antecedents y Consequent)
# ------------------------------
tiene_casa     = ctrl.Antecedent(u_categ, 'tiene_casa')
es_responsable = ctrl.Antecedent(u_categ, 'es_responsable')
creditos_act   = ctrl.Antecedent(u_categ, 'creditos_activos')
salario        = ctrl.Antecedent(u_sal,   'salario_millones')
gastos         = ctrl.Antecedent(u_gast,  'gastos_millones')
prob_prestamo  = ctrl.Consequent(u_prob,  'prob_prestamo')

# ------------------------------
# 4) Funciones de membresía
#    (categóricas: picos en 0 y 1 con tolerancia;
#     numéricas: baja/media/alta)
# ------------------------------
# Categóricas
tiene_casa['no'] = fuzz.trimf(tiene_casa.universe, [0, 0, 0.5])
tiene_casa['si'] = fuzz.trimf(tiene_casa.universe, [0.5, 1, 1])

es_responsable['no'] = fuzz.trimf(es_responsable.universe, [0, 0, 0.5])
es_responsable['si'] = fuzz.trimf(es_responsable.universe, [0.5, 1, 1])

creditos_act['no'] = fuzz.trimf(creditos_act.universe, [0, 0, 0.5])
creditos_act['si'] = fuzz.trimf(creditos_act.universe, [0.5, 1, 1])

# Salario (ajusta a tu política de riesgo)
salario['bajo']   = fuzz.trapmf(salario.universe, [2.0, 2.0, 3.0, 4.0])
salario['medio']  = fuzz.trimf(salario.universe, [3.5, 6.0, 8.0])
salario['alto']   = fuzz.trapmf(salario.universe, [7.0, 8.0, 10.0, 10.0])

# Gastos
gastos['bajos']   = fuzz.trapmf(gastos.universe, [2.0, 2.0, 2.5, 3.0])
gastos['medios']  = fuzz.trimf(gastos.universe, [2.7, 3.5, 4.2])
gastos['altos']   = fuzz.trapmf(gastos.universe, [3.8, 4.3, 5.0, 5.0])

# Salida (probabilidad)
prob_prestamo['baja']  = fuzz.trapmf(prob_prestamo.universe, [1, 1, 20, 40])
prob_prestamo['media'] = fuzz.trimf(prob_prestamo.universe, [30, 50, 70])
prob_prestamo['alta']  = fuzz.trapmf(prob_prestamo.universe, [60, 80, 100, 100])

# ------------------------------
# 5) Reglas difusas (heurísticas ajustables)
# ------------------------------
rules = []
# Ingresos altos + gastos bajos + sin créditos → prob alta
rules.append(ctrl.Rule(salario['alto'] & gastos['bajos'] & creditos_act['no'], prob_prestamo['alta']))
# Ingresos medios + gastos medios + sin créditos → alta (tendiendo a alta)
rules.append(ctrl.Rule(salario['medio'] & gastos['medios'] & creditos_act['no'], prob_prestamo['alta']))
# Ingresos bajos OR gastos altos OR con créditos → prob baja
rules.append(ctrl.Rule(salario['bajo'] | gastos['altos'] | creditos_act['si'], prob_prestamo['baja']))
# Tener casa + sin créditos + no gastos altos → prob alta
rules.append(ctrl.Rule(tiene_casa['si'] & creditos_act['no'] & ~gastos['altos'], prob_prestamo['alta']))
# Responsable del núcleo + ingreso bajo + gastos medios/altos → prob baja
rules.append(ctrl.Rule(es_responsable['si'] & salario['bajo'] & (gastos['medios'] | gastos['altos']), prob_prestamo['baja']))
# Responsable del núcleo + ingreso alto + sin créditos → prob media (prudencia)
rules.append(ctrl.Rule(es_responsable['si'] & salario['alto'] & creditos_act['no'], prob_prestamo['media']))
# Caso intermedio (sin créditos, salario medio, gastos bajos/medios) → media
rules.append(ctrl.Rule(creditos_act['no'] & salario['medio'] & (gastos['bajos'] | gastos['medios']), prob_prestamo['media']))

loan_ctrl = ctrl.ControlSystem(rules)

# ------------------------------
# 6) Función para evaluar una fila
# ------------------------------
def evaluar_prob_prestamo(fila, defuzz='centroid'):
    """
    fila: dict-like con claves:
      - tiene_casa (0/1)
      - es_responsable (0/1)
      - creditos_activos (0/1)
      - salario_millones (float, 2..10)
      - gastos_millones  (float, 2..5)
    defuzz: 'centroid' (default), 'mom', 'som', 'lom', 'bisector'
    """
    sim = ctrl.ControlSystemSimulation(loan_ctrl)
    # Clampeo a universos
    sal = float(np.clip(fila['salario_millones'], u_sal.min(),  u_sal.max()))
    gas = float(np.clip(fila['gastos_millones'],  u_gast.min(), u_gast.max()))
    # Entradas
    sim.input['tiene_casa']       = int(fila['tiene_casa'])
    sim.input['es_responsable']   = int(fila['es_responsable'])
    sim.input['creditos_activos'] = int(fila['creditos_activos'])
    sim.input['salario_millones'] = sal
    sim.input['gastos_millones']  = gas
    # Defuzzificación
    prob_prestamo.defuzzify_method = defuzz
    # Inferencia
    sim.compute()
    return float(sim.output['prob_prestamo'])

# ------------------------------
# 7) Cargar CSV, normalizar categóricas, evaluar y guardar
# ------------------------------
df = pd.read_csv(csv_in)

# Validación de columnas
cols_req = ['tiene_casa','es_responsable','salario_millones','gastos_millones','creditos_activos']
falt = [c for c in cols_req if c not in df.columns]
if falt:
    raise ValueError(f"Faltan columnas: {falt}")

# Mapear si/no -> 0/1
for c in ['tiene_casa','es_responsable','creditos_activos']:
    df[c] = df[c].astype(str).str.strip().map(map_si_no).fillna(0).astype(int)

# Quitar filas con NaN numéricos
df = df.dropna(subset=['salario_millones','gastos_millones']).copy()

# Evaluación fila a fila
probs = []
for _, fila in df.iterrows():
    probs.append(evaluar_prob_prestamo(fila, defuzz='centroid'))

# Agregar columna de salida (limitada 1..100)
df['prob_prestamo'] = np.clip(probs, 1, 100).round(2)

# Guardar CSV con resultados
df.to_csv(csv_out, index=False)
print(f"✅ Listo. 100 clientes procesados.")
print(f"📄 Entrada:  {csv_in}")
print(f"📄 Salida:   {csv_out}")

# Vista rápida
print(df.head(10).to_string(index=False))


✅ Dataset sintético generado: clientes_100.csv
✅ Listo. 100 clientes procesados.
📄 Entrada:  clientes_100.csv
📄 Salida:   clientes_100_con_prob.csv
 tiene_casa  es_responsable  salario_millones  gastos_millones  creditos_activos  prob_prestamo
          1               1              2.41             2.00                 0          50.84
          0               0              6.25             2.09                 1          16.07
          0               1              6.33             2.31                 1          16.07
          0               0              7.10             2.27                 0          56.27
          1               0              7.81             2.70                 0          81.10
          1               1              9.81             3.23                 1          16.07
          1               1              6.13             2.07                 1          16.07
          0               0              4.58             2.00                 0    